<a href="https://colab.research.google.com/github/june-oh/2023_AI_Academy_ASR/blob/main/7_WFST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WFST


## k2-fsa(k2)
-  OpenFST를 개량한 python 기반의 WFST 패키지
- End-to-end 모델과 WFST를 결합할 목적으로 제작됨
- FSA, FST를 tensor 형태로 구현하여 GPU에서 WFST 연산을 가능하게 함
- https://github.com/k2-fsa/k2 

## K2 설치 및 기본 환경 세팅

### Prerequisities (pip 기준)
- Python >= 3.6
- CUDA >= 10.1
- PyTorch == 1.7.1 (conda 설치의 경우 >=1.7.1)

### PyPI (PIP)를 이용한 설치 방법 (권장, but colab에선 비권장)
- pip install k2 (Prerequisities 자동으로 설치됨)

### Colab에서 돌아가는 pip 설치 방법
- ! pip install torch==1.7.1
- ! pip install k2==1.17.dev20220710+cuda10.2.torch1.7.1 -f https://k2-fsa.org/nightly/

### Source code (github)을 이용한 설치 방법
- git clone https://github.com/k2-fsa/k2.git
- cd k2
- python3 setup.py install

In [ ]:
!pip install torch==1.13.1 torchaudio==0.13.1
!pip install k2==1.24.3.dev20230629+cpu.torch1.13.1 -f https://k2-fsa.org/nightly/

In [ ]:
import k2, torch

In [ ]:
import k2.version
k2.version.version.main()

## K2를 이용한 WFST 구현

### Weighted Finite-state Acceptor(WFSA)
- StartState \t EndState \t Symbol(index) \t Weight
- Symbol은 기본적으로 index로 표현/계산됨
- 시각화할 일이 있을 때만 symbol을 표현
- Symbol table 정의가 필요함
- 일반적으로 k2.Fsa.from_str()를 사용하여 생성함
- draw() or to_dot() methods를 이용하여 시각화

In [ ]:
s = '''
0 1 1 1 
0 1 3 5
0 2 1 3
0 2 2 4
0 2 3 7
0 2 4 8
0 2 5 9
1 3 6 9
1 3 5 8
2 3 6 12
3 4 -1 0
4
'''

In [ ]:
sym_str = '''
  <eps> 0
 a 1
 b 2
 c 3
 d 4
 e 5
 f 6
'''

###  Weighted Finite-state Transducer(WFST)
- WFST: StartState \t EndState \t InputSymbol \t OutputSymbol\t Weight
- 시각화된 표현은 arc당 InputSymbol:OutputSymbol/weight
- CTC, lexicon, language model의 최적화된 표현을 위해 
- k2.Fsa.from_str의 acceptor=False option을 이용하여 생성

In [ ]:
s = '''
0 1 1 2 0.1
1 2 2 2 0.3
1 3 2 2 0.4
2 3 1 2 0.5
3 3 1 1 0.6
3 4 -1 -1 0
4
'''
a_fsa = k2.Fsa.from_str(s, acceptor=False)
sym_str = '''
 <eps> 0
 a 1
 b 2
'''
a_fsa.symbols = k2.SymbolTable.from_str(sym_str)
a_fsa.labels_sym = k2.SymbolTable.from_str(sym_str)
a_fsa.aux_labels_sym = k2.SymbolTable.from_str(sym_str)
a_fsa = k2.arc_sort(a_fsa)
a_fsa.draw('fsa_sybmols.svg')

In [ ]:
s = '''
0 1 1 1 1 
0 1 3 3 5
0 2 1 1 3
0 2 2 2 4
0 2 3 3 7
0 2 4 4 8
0 2 5 5 9
1 3 6 6 9
1 3 5 5 8
2 3 6 6 12
3 4 -1 -1 0
4
'''

In [ ]:
sym_str = '''
  <eps> 0
 a 1
 b 2
 c 3
 d 4
 e 5
 f 6
'''

## Composition & Determinization

In [ ]:

s = '''
0 1 1 2 0.1
1 2 2 2 0.3
1 3 2 2 0.4
2 3 1 2 0.5
3 3 1 1 0.6
3 4 -1 -1 0
4
'''

In [ ]:
sym_str = '''
<eps> 0
 a 1
 b 2
 c 3
 d 4
 e 5
 f 6
'''

In [ ]:

s = '''
0 1 2 2 0.1
1 1 2 1 0.2
1 2 1 2 0.3
2 3 2 1 0.5
3 4 -1 -1 0.6
4
'''

In [ ]:
sym_str = '''
<eps> 0
 a 1
 b 2
 c 3
 d 4
 e 5
 f 6
'''